In [125]:
import sys
from importlib import reload
reload(sys)
#sys.setdefaultencoding('utf-8')

import TD_RvNN_exp
import math

import theano
from theano import tensor as T
import numpy as np
from numpy.testing import assert_array_almost_equal

import time
import datetime
import random
from evaluate import *

In [25]:
trainPath = "../resource/rumoureval-2019-training-data.zip"
testPath = "../resource/rumoureval-2019-test-data.zip"

In [26]:
testLabelPath = "../resource/final-eval-key.json" # evaluation file contains test labels

In [27]:
# load data => find the labels for training and test data and put them into a dict structured as needed in RvNN approach
# at first handle zip files
from zipfile import ZipFile
from typing import Dict
def get_archive_directory_structure(archive: ZipFile) -> Dict:
    result = {}
    for file in archive.namelist():
        # Skip directories in archive.
        if file.endswith('/'):
            continue

        d = result
        path = file.split('/')[1:]  # [1:] to skip top-level directory.
        for p in path[:-1]:  # [:-1] to skip filename
            if p not in d:
                d[p] = {}
            d = d[p]
        d[path[-1]] = file
    return result

In [28]:
training_data_archive = ZipFile(trainPath)
training_data_contents = get_archive_directory_structure(
        training_data_archive)

In [29]:
training_data_contents

{'dev-key.json': 'rumoureval-2019-training-data/dev-key.json',
 'reddit-dev-data': {'1jvbd8': {'raw.json': 'rumoureval-2019-training-data/reddit-dev-data/1jvbd8/raw.json',
   'replies': {'cbiol38.json': 'rumoureval-2019-training-data/reddit-dev-data/1jvbd8/replies/cbiol38.json',
    'cbiwlli.json': 'rumoureval-2019-training-data/reddit-dev-data/1jvbd8/replies/cbiwlli.json',
    'cbj4pcw.json': 'rumoureval-2019-training-data/reddit-dev-data/1jvbd8/replies/cbj4pcw.json',
    'cbkixew.json': 'rumoureval-2019-training-data/reddit-dev-data/1jvbd8/replies/cbkixew.json'},
   'source-tweet': {'1jvbd8.json': 'rumoureval-2019-training-data/reddit-dev-data/1jvbd8/source-tweet/1jvbd8.json'},
   'structure.json': 'rumoureval-2019-training-data/reddit-dev-data/1jvbd8/structure.json'},
  '31xv6u': {'raw.json': 'rumoureval-2019-training-data/reddit-dev-data/31xv6u/raw.json',
   'replies': {'cq64vk3.json': 'rumoureval-2019-training-data/reddit-dev-data/31xv6u/replies/cq64vk3.json',
    'cq674h2.json': 

In [30]:
import json

In [31]:
train_labels = json.loads(training_data_archive.read(training_data_contents['train-key.json']))
train_labels

{'subtaskaenglish': {'501760642928635904': 'comment',
  '500270212198174720': 'comment',
  '524971210275565568': 'comment',
  '552836882770690049': 'comment',
  '500289931097296897': 'support',
  '544293230649810944': 'comment',
  '544283032732332032': 'comment',
  '500406679176241153': 'comment',
  '500300182844878848': 'comment',
  '552811591683821568': 'comment',
  '524938343180931073': 'comment',
  '521360786409943041': 'deny',
  '524946532416884736': 'comment',
  '544347456238931968': 'comment',
  '525020658972258305': 'query',
  '552817834628501505': 'comment',
  '524959078229880832': 'comment',
  '553588165408473088': 'comment',
  '544293748377518080': 'comment',
  '553196346375954433': 'query',
  '524957146656821249': 'comment',
  '524992668301262848': 'deny',
  '500294827351621632': 'comment',
  '544307827070615553': 'support',
  '524955418955902976': 'comment',
  '553541495588802560': 'comment',
  '500301494193627136': 'comment',
  '525026551411519488': 'comment',
  '55356514

In [32]:
dev_labels = json.loads(training_data_archive.read(training_data_contents['dev-key.json']))

In [33]:
test_labels = json.load(open(testLabelPath,'r'))
test_labels

{'subtaskaenglish': {'443938194715713536': 'support',
  '774165935041093633': 'support',
  '784071228248109057': 'support',
  '784118929799073793': 'support',
  '784216706080178176': 'support',
  '784316107645603840': 'support',
  '828933929080086528': 'support',
  '830469481621241857': 'support',
  '852844529778601985': 'support',
  '872300483272114177': 'support',
  '872367878871408640': 'support',
  '892781499103277056': 'support',
  '897502435593265152': 'query',
  '897715873145913344': 'support',
  '901939374375370752': 'support',
  '902006921523331073': 'support',
  '902134516575346688': 'support',
  '902224586967621633': 'support',
  '902542822540173314': 'support',
  '902595384635043840': 'support',
  '903336023362547715': 'query',
  '903528113631584256': 'support',
  '904111594912841729': 'support',
  '904200288059494400': 'support',
  '904725019372253184': 'support',
  '904727149747326977': 'support',
  '904762977185808385': 'support',
  '905161219274940416': 'support',
  '90

In [34]:
len(train_labels['subtaskbenglish'])

327

In [35]:
labelDic = {}
for (eid, label) in train_labels['subtaskbenglish'].items():
    if len(eid) == 18: # at first only consider the twitter IDs and labels
        labelDic[eid] = label.lower()

In [36]:
len(labelDic)

297

In [37]:
for (eid, label) in dev_labels['subtaskbenglish'].items():
    if len(eid) == 18: # at first only consider the twitter IDs and labels
        labelDic[eid] = label.lower()

In [38]:
len(labelDic)

325

In [39]:
for (eid, label) in test_labels['subtaskbenglish'].items():
    if len(eid) == 18: # at first only consider the twitter IDs and labels
        labelDic[eid] = label.lower()

In [40]:
len(labelDic)

381

In [41]:
labelDic # contains ALL labels for training, dev and test dataset

{'500288349924782080': 'unverified',
 '500308076004929537': 'unverified',
 '544282227035869184': 'true',
 '529695367680761856': 'false',
 '544324444773433348': 'true',
 '544350712365207552': 'true',
 '500389488217309184': 'unverified',
 '552806309540528128': 'false',
 '552978099357237248': 'false',
 '524956129017995264': 'false',
 '500332933098385408': 'unverified',
 '544271362022338560': 'true',
 '544514564407427072': 'true',
 '553587672137334785': 'true',
 '500363126294863876': 'unverified',
 '553474188259102720': 'unverified',
 '500394061887709184': 'unverified',
 '544309275141885952': 'false',
 '552811386259386370': 'true',
 '524925987239120897': 'true',
 '576323086888361984': 'unverified',
 '544314234541469696': 'true',
 '544381485591982083': 'true',
 '553505242554175489': 'unverified',
 '544278985249550337': 'true',
 '553586897168392192': 'true',
 '524975705206304769': 'true',
 '529653029747064832': 'false',
 '577258317942149120': 'unverified',
 '544367462012432384': 'true',
 '52

In [42]:
# generating the tree from all the zipped data
twitter_english = training_data_contents['twitter-english']

In [43]:
test_data_archive = ZipFile(testPath)
test_data_contents = get_archive_directory_structure(test_data_archive)
twitter_en_test_data = test_data_contents['twitter-en-test-data']

In [44]:
# calculate parent_num, indexP and indexC for the treeDic
def calc_parent_num(tree_branch: Dict) -> int: # go recursively through tree and calculate parent_num
    if isinstance(tree_branch, Dict):
        return 1 + (max(map(calc_parent_num, tree_branch.values())) if tree_branch else 0)
    return 0

In [45]:
def find_parent_node(tree_branch: Dict, indexC: int): # go recursively through the tree and find parent index
    for indexP, sub_branch in tree_branch.items(): # keep indexP and go through the keys
        if indexC in sub_branch: # search for indexC in the keys
            return indexP # if it is found, return the parent node
        elif isinstance(sub_branch, dict): # else check if the keys itself are a dictionary
            parent_node = find_parent_node(sub_branch, indexC) # if so, recursively call again the function with the subtree 
            if parent_node is not None: # return the value only if it exists (otherwise we should add try and catch later)
                return parent_node

In [46]:
# create treeDic
treeDic = {}
for archive, topics in [(training_data_archive, twitter_english.items()),
                            (test_data_archive, twitter_en_test_data.items())]:
    for topic, threads in topics:
            for thread in threads.values():
                # get the source information as a Dict (contains all info about the source post)
                source_information = json.loads(archive.read(list(thread['source-tweet'].values())[0]))
                eid = source_information['id'] # get the source ID
                post_structure = json.loads(archive.read(thread['structure.json'])) # get the thread structure as a Dict
                parent_num = calc_parent_num(post_structure) # calculate the number of reply levels in each thread structure
                print(eid, parent_num,  " PAUSE ")
                indexC = eid # initialize post index with source post index
                if eid not in treeDic: # create empty entry first to make the key accessable
                    treeDic[eid] = {}
                treeDic[eid][indexC] = {'parent':'None', 'parent_num':parent_num}
                if 'replies' in thread: # some "replies" folders seem to be empty and then this for loop would throw an error
                    for reply in thread['replies'].values(): # for every reply post
                        # get the reply information as a Dict (contains all info about the reply post)
                        reply_information = json.loads(archive.read(reply))
                        indexC = reply_information['id'] # indexC = reply_ID, just named indexC for consistency with
                        # RvNN implementation; maybe change variable names later for better understanding
                        # find out parent of each reply node
                        indexP = find_parent_node(post_structure, str(indexC))
                        print(indexC, indexP, parent_num)
                        treeDic[eid][indexC] = {'parent':indexP, 'parent_num':parent_num}

552783667052167168 4  PAUSE 
552785374507175936 552783667052167168 4
552786226546495488 552785374507175936 4
552789345628069888 552786226546495488 4
552792862610694144 552783667052167168 4
552943855021330432 552783667052167168 4
552785375161499649 2  PAUSE 
552785851655393281 552785375161499649 2
552785877748158464 552785375161499649 2
552788073340489728 552785375161499649 2
552788945017516032 10  PAUSE 
552789083211460608 552788945017516032 10
552789147606581248 552788945017516032 10
552789720540119040 552789083211460608 10
552790062778548224 552788945017516032 10
552790386092294144 552788945017516032 10
552790446871945217 552788945017516032 10
552792005559189504 552788945017516032 10
552792919325671425 552788945017516032 10
552793167754715136 552788945017516032 10
552793839782862848 552788945017516032 10
552794223263903744 552789083211460608 10
552794307632332800 552792005559189504 10
552794594115870720 552794307632332800 10
552794881601859584 552792005559189504 10
552795498349101057

552810655657787392 552806309540528128 8
552811106046324736 552808387415900160 8
552811152456318976 552810655657787392 8
552811900686176257 552811106046324736 8
552812099294863362 552808109551263744 8
552812220438945793 552811106046324736 8
552812407064899585 552811900686176257 8
552812425758912512 552811900686176257 8
552812983370268672 552812407064899585 8
552813608694857730 552812425758912512 8
552814267020636160 552812983370268672 8
552814555764506624 552814267020636160 8
552814637696450560 552812983370268672 8
552814686149017600 552813608694857730 8
552817422366154752 552806767625641984 8
552827794976022528 552806767625641984 8
553219824059310080 552811106046324736 8
553221961740857344 553219824059310080 8
553224135334051840 553221961740857344 8
553225926436089856 553224135334051840 8
553226523650437120 553225926436089856 8
552806757672964097 3  PAUSE 
552807298192510977 552806757672964097 3
552807311191048193 552806757672964097 3
552808047643729920 552806757672964097 3
55280813773

553213552337891328 553213293964562434 5
553217526646439936 553203935088214017 5
553891758271369216 553198499891007488 5
553212962044149761 3  PAUSE 
553213078889062401 553212962044149761 3
553213579672178689 553212962044149761 3
553214220364685313 553212962044149761 3
553214228488667136 553212962044149761 3
553214496567590914 553212962044149761 3
553217937117831168 553214496567590914 3
553232132269953024 553214496567590914 3
553221600955621376 2  PAUSE 
553228413922967552 553221600955621376 2
553262052899164161 553221600955621376 2
553461741917863936 4  PAUSE 
553462405800288256 553461741917863936 4
553462961365581824 553461741917863936 4
553463231386497024 553461741917863936 4
553463489209978880 553461741917863936 4
553464272043638785 553461741917863936 4
553467937575952384 553461741917863936 4
553469592488931328 553461741917863936 4
553471188530249730 553461741917863936 4
553496340580556800 553461741917863936 4
553503688946225152 553461741917863936 4
553504117230800896 55346296136558

553552100417110017 553551840428949504 10
553552287889494017 553548567420628992 10
553552395704082432 553548567420628992 10
553552625724301314 553548567420628992 10
553553247953514496 553552036915335169 10
553553255255781376 553548567420628992 10
553553754424115200 553553247953514496 10
553554289835376643 553548567420628992 10
553554620229115905 553553754424115200 10
553555066754695168 553554620229115905 10
553555725084270593 553555066754695168 10
553559162018611201 553548567420628992 10
553559527216656385 553548567420628992 10
553560292635189248 553559162018611201 10
553561410920521729 553560292635189248 10
553562623200210944 553561410920521729 10
553565149790224387 553562623200210944 10
553549686129561600 5  PAUSE 
553549946725875712 553549686129561600 5
553550000584540160 553549686129561600 5
553550346195574784 553549686129561600 5
553550482925715458 553550000584540160 5
553550998497918976 553550000584540160 5
553551156081745920 553550482925715458 5
553551171802382336 553550000584540

553590974044069888 553590459688570880 2
553590981967511553 553590459688570880 2
553591098074226688 553590459688570880 2
553591134807920640 553590459688570880 2
553591179078410240 553590459688570880 2
553591360327282689 553590459688570880 2
553591453675696129 553590459688570880 2
553591555479461888 553590459688570880 2
553591570680004608 553590459688570880 2
553591626485215232 553590459688570880 2
553591653404270592 553590459688570880 2
553591766893727744 553590459688570880 2
553591990856986624 553590459688570880 2
553592018270978049 553590459688570880 2
553590835850514433 6  PAUSE 
553590917773656065 553590835850514433 6
553590931866521602 553590835850514433 6
553591230006067200 553590835850514433 6
553591259009658881 553590835850514433 6
553591266316152832 553590835850514433 6
553591307206422528 553590835850514433 6
553591557325336576 553590835850514433 6
553591583623245824 553590835850514433 6
553591773537513473 553590835850514433 6
553591850532347906 553590835850514433 6
55359223335

...

544480133466574848 544476808566276097 8
544480379147542528 544476808566276097 8
544480395585392641 544476808566276097 8
544480456478314496 544476808566276097 8
544480465982611458 544476808566276097 8
544482249459388416 544476808566276097 8
544482468410044417 544476808566276097 8
544482919595511808 544482249459388416 8
544483162965807106 544482919595511808 8
544483799174627329 544483162965807106 8
544484075994509312 544483799174627329 8
544485799149187072 544477183851630592 8
544500025016197121 544485799149187072 8
544501395219808256 544480465982611458 8
544501573624545280 544480465982611458 8
544505605567041536 544500025016197121 8
544531648742973440 544480107872915456 8
544991347304714240 544505605567041536 8
544996195949871104 544991347304714240 8
544996807324213249 544996195949871104 8
545017860754792448 544996195949871104 8
544491151118860289 4  PAUSE 
544491585384505344 544491151118860289 4
544491587271544832 544491151118860289 4
544492317546393600 544491587271544832 4
54449310996

940239828821651457 940078473061609472 2
940213985563037697 940078473061609472 2
940109966261628928 940078473061609472 2
940192849332768770 940078473061609472 2
941727586765492224 940078473061609472 2
940246766334676992 940078473061609472 2
940115108352606208 940078473061609472 2
940839899019862018 940078473061609472 2
904762977185808385 2  PAUSE 
904791420376055810 904762977185808385 2
904763799588175872 904762977185808385 2
904774032117178374 904762977185808385 2
904772403036913666 904762977185808385 2
905096646064619521 904762977185808385 2
904769564034125824 904762977185808385 2
904793768171274240 904762977185808385 2
904768844333432832 904762977185808385 2
904768514459811840 904762977185808385 2
904763943557718016 904762977185808385 2
904773488178860033 904762977185808385 2
904766093671243776 904762977185808385 2
904792927276294144 904762977185808385 2
905094471372627968 904762977185808385 2
904789817405444096 904762977185808385 2
904775690351980547 904762977185808385 2
90476841888

914751235709956096 914684276477710336 2
914836385944756224 914684276477710336 2
915374641148858368 914684276477710336 2
915029664699166723 914684276477710336 2
914851859050958853 914684276477710336 2
914684769832546309 914684276477710336 2
914686391950462977 914684276477710336 2
915075077745577984 914684276477710336 2
914582144952995840 2  PAUSE 
914828408223420416 914582144952995840 2
915694727885754368 914582144952995840 2
914632109465260033 914582144952995840 2
914717424598917121 914582144952995840 2
784118929799073793 2  PAUSE 
784519106103955456 784118929799073793 2
784119184825483264 784118929799073793 2
784120792955117568 784118929799073793 2
784130641885556736 784118929799073793 2
784132406076907520 784118929799073793 2
784144643839623168 784118929799073793 2
784128534759563264 784118929799073793 2
784156802862768128 784118929799073793 2
784132637283717123 784118929799073793 2
784121303401889793 784118929799073793 2
784316107645603840 2  PAUSE 
784360749489676288 78431610764560

934868212528943105 934828842505723904 2
940455814594203648 934828842505723904 2
935080042773864448 934828842505723904 2
934862953383432193 934828842505723904 2
943600096838672384 934828842505723904 2
934875827711471616 934828842505723904 2
935751277694345217 934828842505723904 2
955751746177028096 934828842505723904 2
934849500635688961 934828842505723904 2
934992964371255296 934828842505723904 2
935036822035542016 934828842505723904 2
934848346853920768 934828842505723904 2
935477420806868992 934828842505723904 2
934892756962545664 934828842505723904 2
941305217454403584 2  PAUSE 
941389979816812544 941305217454403584 2
941314271585210369 941305217454403584 2
941601279440244736 941305217454403584 2
443938194715713536 2  PAUSE 
443939406823436288 443938194715713536 2
443941497541754880 443938194715713536 2
443941090115088384 443938194715713536 2
443939729071820801 443938194715713536 2
443940322209329152 443938194715713536 2
443953494765367296 443938194715713536 2
443940443219570689 443

902552099048841216 902542822540173314 2
902542921118994434 902542822540173314 2
902546252025520129 902542822540173314 2
902746794576183296 902542822540173314 2
902595384635043840 2  PAUSE 
902598683358875648 902595384635043840 2
902603984967319552 902595384635043840 2
902604546349858816 902595384635043840 2
902598626039304192 902595384635043840 2
902600332169097216 902595384635043840 2
904200288059494400 2  PAUSE 
904208233312555008 904200288059494400 2
904214103039762432 904200288059494400 2
904397590497681408 904200288059494400 2
904201520450859008 904200288059494400 2
904236294620016640 904200288059494400 2
904330584033554435 904200288059494400 2
904221611498209280 904200288059494400 2
904201281790767105 904200288059494400 2
904209561342533637 904200288059494400 2
904200505747955713 904200288059494400 2
904208420118695936 904200288059494400 2
904200520172134401 904200288059494400 2
904200934493958144 904200288059494400 2
904200672668725248 904200288059494400 2
904212968845570048 904

In [47]:
treeDic

{552783667052167168: {552783667052167168: {'parent': 'None', 'parent_num': 4},
  552785374507175936: {'parent': '552783667052167168', 'parent_num': 4},
  552786226546495488: {'parent': '552785374507175936', 'parent_num': 4},
  552789345628069888: {'parent': '552786226546495488', 'parent_num': 4},
  552792862610694144: {'parent': '552783667052167168', 'parent_num': 4},
  552943855021330432: {'parent': '552783667052167168', 'parent_num': 4}},
 552785375161499649: {552785375161499649: {'parent': 'None', 'parent_num': 2},
  552785851655393281: {'parent': '552785375161499649', 'parent_num': 2},
  552785877748158464: {'parent': '552785375161499649', 'parent_num': 2},
  552788073340489728: {'parent': '552785375161499649', 'parent_num': 2}},
 552788945017516032: {552788945017516032: {'parent': 'None', 'parent_num': 10},
  552789083211460608: {'parent': '552788945017516032', 'parent_num': 10},
  552789147606581248: {'parent': '552788945017516032', 'parent_num': 10},
  552789720540119040: {'pare

In [49]:
len(treeDic)

381

In [50]:
labelDic

{'500288349924782080': 'unverified',
 '500308076004929537': 'unverified',
 '544282227035869184': 'true',
 '529695367680761856': 'false',
 '544324444773433348': 'true',
 '544350712365207552': 'true',
 '500389488217309184': 'unverified',
 '552806309540528128': 'false',
 '552978099357237248': 'false',
 '524956129017995264': 'false',
 '500332933098385408': 'unverified',
 '544271362022338560': 'true',
 '544514564407427072': 'true',
 '553587672137334785': 'true',
 '500363126294863876': 'unverified',
 '553474188259102720': 'unverified',
 '500394061887709184': 'unverified',
 '544309275141885952': 'false',
 '552811386259386370': 'true',
 '524925987239120897': 'true',
 '576323086888361984': 'unverified',
 '544314234541469696': 'true',
 '544381485591982083': 'true',
 '553505242554175489': 'unverified',
 '544278985249550337': 'true',
 '553586897168392192': 'true',
 '524975705206304769': 'true',
 '529653029747064832': 'false',
 '577258317942149120': 'unverified',
 '544367462012432384': 'true',
 '52

In [54]:
train_IDs, test_IDs = [], []

In [72]:
for (eid, label) in train_labels['subtaskbenglish'].items():
    if len(eid) == 18: # for now only consider the twitter IDs and labels
        train_IDs.append(eid)
for (eid, label) in dev_labels['subtaskbenglish'].items():
    if len(eid) == 18: # for now only consider the twitter IDs and labels
        train_IDs.append(eid)
for (eid, label) in test_labels['subtaskbenglish'].items():
    if len(eid) == 18: # for now only consider the twitter IDs and labels
        test_IDs.append(eid)

In [117]:
# load label function
def loadLabel(label, l1, l2, l3):
    labelset_f, labelset_t, labelset_u = ['false'], ['true'], ['unverified']
    if label in labelset_f:
       y_train = [1,0,0]
       l1 += 1
    if label in labelset_t:
       y_train = [0,1,0]
       l2 += 1
    if label in labelset_u:
       y_train = [0,0,1]
       l3 += 1
    return y_train, l1,l2,l3

In [127]:
def constructTree(tree):
    ## tree: {index1:{'parent':, 'maxL':, 'vec':}
    ## 1. ini tree node
    index2node = {}
    for i in tree:
        node = TD_RvNN_exp.Node_tweet(idx=i)
        index2node[i] = node
    ## 2. construct tree
    for j in tree:
        indexC = j 
        indexP = tree[j]['parent']
        nodeC = index2node[indexC]
        #wordFreq, wordIndex = str2matrix( tree[j]['vec'], tree[j]['maxL'] )
        #print tree[j]['maxL']
        #nodeC.index = wordIndex
        #nodeC.word = wordFreq
        #nodeC.time = tree[j]['post_t']
        ## not root node ## 
        if not indexP == 'None':
           nodeP = index2node[int(indexP)]
           nodeC.parent = nodeP
           nodeP.children.append(nodeC)
        ## root node ##
        else:
           root = nodeC
    ## 3. convert tree to DNN input    
    parent_num = tree[j]['parent_num'] 
    #ini_x, ini_index = str2matrix( "0:0", tree[j]['maxL'] )
    #x_word, x_index, tree = tree_gru_u2b.gen_nn_inputs(root, ini_x, ini_index) 
    tree = TD_RvNN_exp.gen_nn_inputs(root) 
    return tree, parent_num

In [129]:
tree_train, word_train, index_train, y_train, parent_num_train, c = [], [], [], [], [], 0
l1,l2,l3 = 0,0,0
for eid in train_IDs:
    if eid not in labelDic: continue
    if int(eid) not in treeDic: continue
    if len(treeDic[int(eid)]) <= 0:
        continue
    label = labelDic[str(eid)] # somehow eid must be referenced as string to adress the key
    y, l1,l2,l3 = loadLabel(label, l1, l2, l3)
    y_train.append(y)
    tree, parent_num = constructTree(treeDic[int(eid)])

OverflowError: Python int too large to convert to C long